In [216]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [39]:
df = pd.read_parquet("DSCB310 - UE2 - Shopping Carts.parquet")

In [40]:
df.reset_index(drop= True).head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county
0,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo
1,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo
2,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo
3,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo
4,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo


In [41]:
from urllib.request import urlopen
import json
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    geojsn = json.load(response)

In [42]:
a = geojsn["features"].__len__()
for i in range(a,0,-1):
    if geojsn["features"][i-1]["properties"]["STATE"] != "06":
        geojsn["features"].pop(i-1)

### **P2:** Lassen sich regionale Unterschiede im Trinkgeldverhalten erkennen?

In [43]:
df_county_tip1 = df.groupby(["county", "order_id"]).agg(tip = ("tip", "mean")).reset_index()

In [44]:
df_county_tip2 = df_county_tip1.groupby("county").agg(tip = ("tip", "mean"), orders= ("order_id", "count")).sort_values(by= "tip", ascending= False).reset_index()

In [45]:
df_county_tip2["orders"] = (df_county_tip2.orders / df_county_tip2.orders.sum()) # *10

In [46]:
df_county_tip2.head()

,county,tip,orders
0,San Diego,0.530895,0.003920
1,Santa Barbara,0.507548,0.023030
2,Lassen,0.502494,0.003964
3,Del Norte,0.501239,0.003323
4,Modoc,0.492131,0.015599


In [47]:
(df_county_tip2.loc[df_county_tip2.county == "Glenn"].orders.values[0] * 100).round(3)

5.856

In [279]:
px.colors.sequential.swatches()

In [289]:
fig1 = make_subplots(cols= 1, rows= 2, shared_xaxes= True, shared_yaxes= False, vertical_spacing= 0)

for county in df_county_tip2.county:

 if df_county_tip2[df_county_tip2.county == county].tip.values >= 0.5:
  color = px.colors.sequential.Purpor[5]
  text= df_county_tip2.loc[df_county_tip2.county == county].tip.values.round(3)

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].tip,
    name= f"{county}: {(df_county_tip2.loc[df_county_tip2.county == county].tip.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{county}" 
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].orders,
    name= f"Orders: {(df_county_tip2.loc[df_county_tip2.county == county].orders.values[0] * 100).round(3)}%",
    base = -df_county_tip2.loc[df_county_tip2.county == county].orders,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{county}"
   ),
   col= 1,
   row= 2
  )

 elif 0.4 < df_county_tip2.loc[df_county_tip2.county == county].tip.values < 0.5:
  color= px.colors.sequential.Greys[3]

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].tip,
    name= f"{county}: {(df_county_tip2.loc[df_county_tip2.county == county].tip.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{county}"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].orders,
    name= f"Orders: {(df_county_tip2.loc[df_county_tip2.county == county].orders.values[0] * 100).round(3)}%",
    base = -df_county_tip2.loc[df_county_tip2.county == county].orders,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{county}"
   ),
   col= 1,
   row= 2
  )

 elif 0.3 < df_county_tip2.loc[df_county_tip2.county == county].tip.values < 0.4:
  color= px.colors.sequential.Greys[2]

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].tip,
    name= f"{county}: {(df_county_tip2.loc[df_county_tip2.county == county].tip.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{county}"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].orders,
    name= f"Orders: {(df_county_tip2.loc[df_county_tip2.county == county].orders.values[0] * 100).round(3)}%",
    base = -df_county_tip2.loc[df_county_tip2.county == county].orders,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{county}"
   ),
   col= 1,
   row= 2
  )

 elif 0.2 < df_county_tip2.loc[df_county_tip2.county == county].tip.values < 0.3:
  color= px.colors.sequential.Greys[1]
  text= df_county_tip2.loc[df_county_tip2.county == county].tip.values.round(3)

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].tip,
    name= f"{county}: {(df_county_tip2.loc[df_county_tip2.county == county].tip.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{county}"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].orders,
    name= f"Orders: {(df_county_tip2.loc[df_county_tip2.county == county].orders.values[0] * 100).round(3)}%",
    base = -df_county_tip2.loc[df_county_tip2.county == county].orders,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{county}"
   ),
   col= 1,
   row= 2
  )

fig1.add_hline(
 y= 0.5,
 opacity= 0.3,
 line_dash= "solid",
 annotation_text= 0.5,
 annotation_position= "top right",
 line_color= px.colors.sequential.Greys[7],
 annotation_font_color= px.colors.sequential.Greys[7],
 annotation_font_size= 12,
 layer= "below",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= 0.3,
 opacity= 0.3,
 line_dash= "solid",
 annotation_text= 0.3,
 annotation_position= "top right",
 line_color= px.colors.sequential.Greys[7],
 annotation_font_color= px.colors.sequential.Greys[7],
 annotation_font_size= 12,
 layer= "below",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= -0.05,
 opacity= 0.3,
 line_dash= "solid",
 annotation_text= -0.05,
 annotation_position= "top right",
 line_color= px.colors.sequential.Greys[7],
 annotation_font_color= px.colors.sequential.Greys[7],
 annotation_font_size= 12,
 layer= "below",
 col= 1,
 row= 2
)

fig1.add_hline(
 y= -0.03,
 opacity= 0.3,
 line_dash= "solid",
 annotation_text= -0.03,
 annotation_position= "top right",
 line_color= px.colors.sequential.Greys[7],
 annotation_font_color= px.colors.sequential.Greys[7],
 annotation_font_size= 12,
 layer= "below",
 col= 1,
 row= 2
)

fig1.add_hline(
 y= 0,
 opacity= 1,
 line_dash= "solid",
 line_color= "black",
 layer= "below",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= 0,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 layer= "below",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 layer= "below",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 layer= "below",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= -0.075,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 layer= "below",
 col= 1,
 row= 2
)

fig1.update_annotations(opacity= 0.8)
fig1.update_layout(title= dict(text= "<b>Regionale Unterschiede im Trinkgeldverhalten</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), legend_title_text = "<b>Counties</b>", height= 700)
fig1.update_xaxes(title_text="<b>Counties</b>", ticks= "outside", col= 1, row= 2, showline= True)
fig1.update_xaxes(showline= False, col= 1, row= 1)
fig1.update_yaxes(title_text="<b>Tipp Wshk.</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 1, tickvals= [0, 0.1, 0.2, 0.3, 0.4, 0.5], showline= False)
fig1.update_yaxes(title_text="<b>Order Anteil</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 2, tickvals= [0, -0.01, -0.03, -0.05, -0.07])
fig1.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))
fig1.show()

In [290]:

colorscales = [(0,px.colors.sequential.Greys[0]),
                (0.3, px.colors.sequential.Greys[2]),
                (0.6, px.colors.sequential.Purp[3]),
                (0.8, px.colors.sequential.Purp[5]),
                (1,px.colors.sequential.Purpor[5])]


#fig = make_subplots(cols= 2, rows=2, specs= [[dict(type= "table"), dict(type= "choroplethmapbox", rowspan = 2)], [dict(type= "table"), None]])
fig = make_subplots(cols= 1, rows= 2, specs = [[dict(type= "choroplethmapbox")], [dict(type= "table")]], vertical_spacing = 0.03, shared_xaxes= True)

fig.add_trace(
    go.Choroplethmapbox(
        geojson= geojsn,
        featureidkey= "properties.NAME",
        locations= df_county_tip2.county,
        z= df_county_tip2.tip,
        showlegend=False,
        showscale=True,
        marker= dict(line= dict(width= 0.2, color= "black"), opacity= 1),
        colorscale = colorscales
    ),
    col= 1,
    row= 1
)

fig.add_trace(
    go.Table(
        columnorder= [1, 2, 3, 4, 5],
        columnwidth= [1, 1, 1, 1, 1],
        header= dict(
            values= list(df_county_tip2.columns),
            fill_color= px.colors.sequential.Purp[3],
            line_color= px.colors.sequential.Greys[4],
            font= dict(color= "white", size=15),
            height= 35
        ),
        cells= dict(
            values= df_county_tip2.round(3).T.values,
            line_color= px.colors.sequential.Greys[4],
            font= dict(color= "black", size=13),
            align= "center",
            height= 30
        )
    ),
    col= 1,
    row= 2
)


# fig.update_layout(
#     updatemenus= [
#             dict(buttons= list([
#                 dict(
#                     label= county,
#                     method= "update",
#                     args= [
#                         dict(
#                             visible= (np.append([True], np.append([True],df.county.unique() == county))),
#                             z = [df_county_tip2.loc[df_county_tip2.county == county].tip],
#                             locations= [df_county_tip2.loc[df_county_tip2.county == county].county],
#                             cells = dict(
#                                         values = df_county_tip2.loc[df_county_tip2.county == county].round(3).T.values,
#                                         height= 30,
#                                         font= dict(color= "black", size=13)
#                                     )
#                             )
#                     ]
#                 )
#                 for county in df_county_tip2.county.unique().tolist()
#             ]),
#             direction= "down",
#             xanchor= "left",
#             x= 0.45,
#             yanchor= "top",
#             y= 1,
#             showactive= True
#         ),
#     ]
# )

fig.update_layout(mapbox_style="carto-positron",
                    mapbox_zoom= 5.1,
                    mapbox_center = {"lat": 37, "lon": -120},
                    height= 700,
                    title= dict(text= "<b>Regionale Unterschiede im Trinkgeldverhalten</b>",
                                font_size= 20,
                                y= 1,
                                x= 0.45
                            )
                )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [292]:
px.colors.sequential.swatches()